In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as Calendar 
import pytz

In [2]:
def is_dst(start_time, end_time, timezone='America/New_York'):
    # localize times to the given timezone
    tz = pytz.timezone(timezone)
    start_local = tz.localize(start_time)
    end_local = tz.localize(end_time)

    # get UTC offsets in hours
    start_offset = start_local.utcoffset().total_seconds() / 3600
    end_offset = end_local.utcoffset().total_seconds() / 3600
    
    # check if offsets differ, which indicates a DST change
    if start_offset != end_offset:
        if end_offset > start_offset: # start is one hour shorter 
            return -1 # DST start
        else:
            return 1
    return 0

def is_valid_hour(peak_name, curr_time, holidays_arr, valid_times):
    weekday = curr_time.weekday()
    date = curr_time.floor('d')

    # check for peaktype conditions for different weekday
    if (peak_name == 'onpeak' or peak_name == 'offpeak') and (weekday >= 5 or date in holidays_arr):
        return False
    if peak_name == '2x16H' and (weekday < 5 and date not in holidays_arr):
        return False
    
    # check if the current hour falls within any valid interval
    hour =  curr_time.hour
    for interval in valid_times:
        start_hr, end_hr = interval
        if start_hr <= hour < end_hr:
            return True
    
    return False

In [3]:

def get_hours(iso, peak_type, period):
    peak_hours = {
        'onpeak': [(6, 22)], # non-holiday weekday
        'offpeak': [(0, 6), (22, 24)], # non-holiday weekday
        'flat': [(0, 24)], # every day
        '2x16H': [(6, 22)], # weekend and holiday
        '7x8': [(0, 6), (22, 24)] # every day
    }
    
    # parse the string 'period' to get start and end dates
    year = int(period[:4])
    if period[-1] == 'A':
        start_date = pd.Timestamp(f'{year}-1-1')
        actual_end_date = pd.Timestamp(f'{year}-12-31')
    elif period[-2] == 'Q':
        quarter = int(period[-1])
        mo = 3 * (quarter - 1) + 1
        start_date = pd.Timestamp(f'{year}-{mo}-1')
        actual_end_date = start_date + pd.offsets.QuarterEnd()
    elif period[-1].isalpha():
        mo = period[-3:]
        start_date =  pd.Timestamp(f'{year}-{mo}-1')
        actual_end_date = start_date + pd.offsets.MonthEnd()
    else:
        start_date = pd.Timestamp(period)
        actual_end_date = start_date

    end_date = actual_end_date + pd.DateOffset(1)

    # adjust for DST if needed
    dst = 0
    if iso != 'MISO' and (peak_type == 'offpeak' or peak_type == 'flat' or peak_type == '7x8') :
        dst += is_dst(start_date, end_date)
    
    # find the holidays within the period
    cal = Calendar()
    holidays = cal.holidays(start=start_date, end=end_date)

    # generate all hours in the period
    all_date_hours = pd.date_range(start_date, end_date, freq='h')

    valid_times = peak_hours[peak_type]
    num_hours = 0

    # count valid hours
    for an_hour in all_date_hours[:-1]:
        if is_valid_hour(peak_type, an_hour, holidays, valid_times):
            num_hours += 1

    return {
        'iso': iso,
        'peak_type': peak_type.upper(),
        'startdate': start_date.strftime('%Y-%m-%d'),
        'enddate': actual_end_date.strftime('%Y-%m-%d'),
        'num_hours': num_hours + dst
    }

In [4]:
# check for sample run

results = get_hours("ERCOT", "onpeak", "2019May")
print(results)

{'iso': 'ERCOT', 'peak_type': 'ONPEAK', 'startdate': '2019-05-01', 'enddate': '2019-05-31', 'num_hours': 352}


In [5]:
# test cases on different peak types

example_onpeak = get_hours("ERCOT", "onpeak", "2019May")
example_offpeak = get_hours("ERCOT", "offpeak", "2019May")
example_flat = get_hours("ERCOT", "flat", "2019May")
example_216 = get_hours("ERCOT", "2x16H", "2019May")
example_78 = get_hours("ERCOT", "7x8", "2019May")

print(example_onpeak)
print(example_offpeak)
print(example_flat)
print(example_216)
print(example_78)

{'iso': 'ERCOT', 'peak_type': 'ONPEAK', 'startdate': '2019-05-01', 'enddate': '2019-05-31', 'num_hours': 352}
{'iso': 'ERCOT', 'peak_type': 'OFFPEAK', 'startdate': '2019-05-01', 'enddate': '2019-05-31', 'num_hours': 176}
{'iso': 'ERCOT', 'peak_type': 'FLAT', 'startdate': '2019-05-01', 'enddate': '2019-05-31', 'num_hours': 744}
{'iso': 'ERCOT', 'peak_type': '2X16H', 'startdate': '2019-05-01', 'enddate': '2019-05-31', 'num_hours': 144}
{'iso': 'ERCOT', 'peak_type': '7X8', 'startdate': '2019-05-01', 'enddate': '2019-05-31', 'num_hours': 248}


In [6]:
# MISO vs other ISO type with DST adjustment

example_1 = get_hours("MISO", "flat", "2023Mar")
example_2 = get_hours("ERCOT", "flat", "2023Mar")
print(example_1)
print(example_2)

{'iso': 'MISO', 'peak_type': 'FLAT', 'startdate': '2023-03-01', 'enddate': '2023-03-31', 'num_hours': 744}
{'iso': 'ERCOT', 'peak_type': 'FLAT', 'startdate': '2023-03-01', 'enddate': '2023-03-31', 'num_hours': 743}
